### 라이브러리 불러오기

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm.auto import tqdm
import time

### 접속 및 불변 요소 가져오기

In [2]:
url = "https://cs.nexon.com/HelpBoard/Nexon?gamecode=18"
driver = webdriver.Chrome()
driver.get(url)
time.sleep(0.5)

view_helplist_btn = driver.find_element(By.CLASS_NAME, 'view_helplist')

all_view = driver.find_element(By.CLASS_NAME, "category_list")
content = driver.find_element(By.CLASS_NAME, "help_content_list")
page = driver.find_element(By.ID, "paging")
next_btn = page.find_element(By.ID, "next")
tot_len_dd = len(all_view.find_elements(By.TAG_NAME, "dd"))

### 주제 id 가져오기

In [3]:
view_helplist_btn.click()

path_list = []
li_list = all_view.find_elements(By.TAG_NAME, "li") # li(strong) - dl(span) - dd(a)
for li in li_list:
    li_text = li.find_element(By.TAG_NAME, "strong").text
    dl_list = li.find_elements(By.TAG_NAME, "dl")
    for dl in dl_list:
        dl_text = dl.find_element(By.TAG_NAME, "span").text
        a_list = dl.find_elements(By.TAG_NAME, "a")
        for a in a_list:
            a_text = a.text
            _id = a.get_attribute("id")
            path_list.append((li_text, dl_text, a_text, _id))

cls_btn = driver.find_element(By.CLASS_NAME, "close")
cls_btn.click()

### 각 주제 게시글 접근하여 제목,내용 가져오기

In [4]:
body_list = []

# for path_id in tqdm([path_list[0], path_list[2], path_list[1]]):
for path_id in tqdm(path_list[0:]):
    # 전체보기
    view_helplist_btn.click()

    # 하나클릭
    _id = path_id[-1]
    a_btn = all_view.find_element(By.ID, _id)
    a_btn.click()

    # 페이지 갯수
    hidden = page.find_element(By.CLASS_NAME, "paging.pagingLi.nexon-hide")
    num_page = int(hidden.get_attribute("order"))

    for i in range(1, num_page):
        dt_list = content.find_elements(By.TAG_NAME, 'dt')
        con_list = content.find_elements(By.CLASS_NAME, 'help_content')
        for dt, con in zip(dt_list[1:], con_list[1:]):
            title = dt.find_element(By.TAG_NAME, 'span').text
            dt.click()
            body = con.text
            body_list.append((*path_id, f"page_{i}", title, body))
            dt.click()
        
        next_btn.click()

  0%|          | 0/78 [00:00<?, ?it/s]

### dict로 정리

In [5]:
data_dic = {}
for body in body_list:
    title = body[5]
    data_dic[title] = {}
    data_dic[title]['body'] = body[6]
    data_dic[title]['meta-data'] = ' -> '.join([body[0], body[1], body[2], body[4], ])

### json 변환

In [7]:
import json

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data_dic, f, indent=2, ensure_ascii=False)